In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/12/10 17:47:16 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 192.168.3.15 instead (on interface enp5s0)
23/12/10 17:47:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.csv("../dados/Posicao_Estoque_Brasil.csv", sep=";", quote="'", header=True, inferSchema=True)
df.printSchema()

root
 |-- CO_UF: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- CO_MUNICIPIO_IBGE: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- CO_CNES: string (nullable = true)
 |-- DT_POSICAO_ESTOQUE: timestamp (nullable = true)
 |-- CO_CATMAT: string (nullable = true)
 |-- DS_PRODUTO: string (nullable = true)
 |-- QT_ESTOQUE: string (nullable = true)
 |-- NU_LOTE: string (nullable = true)
 |-- DT_VALIDADE: string (nullable = true)
 |-- TP_PRODUTO: string (nullable = true)
 |-- SG_PROGRAMA_SAUDE: string (nullable = true)
 |-- DS_PROGRAMA_SAUDE: string (nullable = true)
 |-- SG_ORIGEM: string (nullable = true)



In [4]:
df.head(5)

[Row(CO_UF='11', SG_UF='RO', CO_MUNICIPIO_IBGE='110001', NO_MUNICIPIO="'ALTA FLORESTA D''OESTE'", CO_CNES='2497433', DT_POSICAO_ESTOQUE=datetime.datetime(2023, 12, 9, 0, 0), CO_CATMAT='BR0230570', DS_PRODUTO='SACO PLÁSTICO LIXO, CAPACIDADE 30, COR BRANCA, LARGURA 59, ALTURA 62, CARACTERÍSTICAS ADICIONAIS LEITOSO SUPER RESISTENTE, APLICAÇÃO COLETA DE RESÍDUOS DE SERVIÇOS DE SAÚDE', QT_ESTOQUE='1', NU_LOTE='01022020', DT_VALIDADE='2025-01-30 00:00:00.000', TP_PRODUTO='O', SG_PROGRAMA_SAUDE='HOSP', DS_PROGRAMA_SAUDE='HOSPITALAR', SG_ORIGEM='HORUS'),
 Row(CO_UF='11', SG_UF='RO', CO_MUNICIPIO_IBGE='110001', NO_MUNICIPIO="'ALTA FLORESTA D''OESTE'", CO_CNES='2497433', DT_POSICAO_ESTOQUE=datetime.datetime(2023, 12, 9, 0, 0), CO_CATMAT='BR0233632U0062', DS_PRODUTO='ÓLEO MINERAL (PETROLATO) SOLUÇÃO ORAL 100 ML', QT_ESTOQUE='3', NU_LOTE='0035', DT_VALIDADE='2024-08-30 00:00:00.000', TP_PRODUTO='B', SG_PROGRAMA_SAUDE='AF', DS_PROGRAMA_SAUDE='ASSISTÊNCIA FARMACÊUTICA', SG_ORIGEM='HORUS'),
 Row(CO_U

In [5]:
mun_distinct = df.select("co_municipio_ibge").distinct().count()
print("Quantidade de municípios distintos: ", mun_distinct)

cnes_distinct = df.select("co_cnes").distinct().count()
print("Quantidade de CNES distintos: ", cnes_distinct)

produto_distinct = df.select("co_catmat").distinct().count()
print("Quantidade de produtos distintos: ", produto_distinct)

avg_estabelecimento = cnes_distinct / mun_distinct
print("Média de estabelecimentos por município: ", avg_estabelecimento)

Quantidade de municípios distintos:  4325


Quantidade de CNES distintos:  17503


Quantidade de produtos distintos:  14608
Média de estabelecimentos por município:  4.0469364161849715


In [6]:
from pyspark.sql import functions as F

# Agrupa o DataFrame por co_cnes e conta o número de co_catmat distintos em cada grupo
df_cnes = df.groupBy("co_cnes").agg(F.countDistinct("co_catmat").alias("count_distinct_catmat"))

df_cnes.show()

23/12/10 17:47:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/10 17:47:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/10 17:47:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/10 17:47:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------+---------------------+
|co_cnes|count_distinct_catmat|
+-------+---------------------+
|9478000|                 1355|
|2331748|                  627|
|9273816|                  276|
|7252307|                  124|
|9374787|                  313|
|2341484|                  107|
|4008952|                   22|
|2366991|                   42|
|2415089|                   71|
|5196337|                  435|
|9459634|                  322|
|7164246|                  109|
|7298579|                  152|
|8014949|                  173|
|3009564|                  664|
|2679485|                  319|
|7373759|                  139|
|6595057|                  512|
|2007940|                  191|
|9306471|                  304|
+-------+---------------------+
only showing top 20 rows



In [7]:
df_cnes.describe().show()

23/12/10 17:47:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/10 17:47:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/10 17:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/10 17:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/10 17:47:53 ERROR Executor: Exception in task 25.0 in stage 27.0 (TID 256)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.io.BufferedOutputStream.<init>(BufferedOutputStream.java:75)
	at org.apache.spark.storage.DiskBlockObjectWriter$ManualCloseBufferedOutputStream$1.<init>(DiskBlockObjectWriter.scala:150)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:152)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:161)
	at org.apache.spa

[38,373s][warning][gc,alloc] Executor task launch worker for task 20.0 in stage 27.0 (TID 251): Retried waiting for GCLocker too often allocating 256 words
[38,381s][warning][gc,alloc] Executor task launch worker for task 25.0 in stage 27.0 (TID 256): Retried waiting for GCLocker too often allocating 4098 words


23/12/10 17:47:54 WARN TaskSetManager: Lost task 26.0 in stage 27.0 (TID 257) (192.168.3.15 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 25 in stage 27.0 failed 1 times, most recent failure: Lost task 25.0 in stage 27.0 (TID 256) (192.168.3.15 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.io.BufferedOutputStream.<init>(BufferedOutputStream.java:75)
	at org.apache.spark.storage.DiskBlockObjectWriter$ManualCloseBufferedOutputStream$1.<init>(DiskBlockObjectWriter.scala:150)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:152)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:161)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:308)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:171)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWr

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:


print("Primeiro quartil: ", quartis[0])
print("Segundo quartil (mediana): ", quartis[1])
print("Terceiro quartil: ", quartis[2])